## Imports

In [0]:
!pip install split-folders

In [0]:
import os
from shutil import copyfile

# google drive
from google.colab import drive

import split_folders

## Copy files

Mount Google Drive

In [0]:
drive.mount('/content/gdrive')

Copy tar file from: https://web.inf.ufpr.br/vri/databases/breast-cancer-histopathological-database-breakhis/

In [0]:
!tar -xzvf "/content/gdrive/My Drive/colab_data/BreaKHis_v1.tar.gz"

## Create Empty Directories

create directories to store formatted :

In [0]:
BASE_PATH = "/content/BreaKHis_v1/histology_slides/breast/"

DATA_ROOT = "/content/breakhis-ip/"

ZOOMS = ['40X', '100X', '200X', '400X']

CLASSES = ['benign', 'malignant']

# Seperate 40x, 100x, 200x, 400x images into seperate directories. 
# Each element of the below list is a path to the directory.
directories = [DATA_ROOT + "/" + x for x in ZOOMS]

# create directories
for directory in directories:
    # Make benign and malignant sub-directories for each directory if they do not exist
    for suffix in ["/" + x for x in CLASSES]:
      if not os.path.exists(directory + suffix):
          os.makedirs(directory + suffix)

## Copy and Split

Copy files under appropriate folders:

In [0]:
for class_type in CLASSES:
  sub_path = BASE_PATH + "/" + class_type + "/SOB"
  sub_types = os.listdir(sub_path)
  sub_types = [x for x in sub_types if x!= '.DS_Store']
  for sub_type in sub_types:
    patient_path = sub_path + "/" + sub_type
    patients = os.listdir(patient_path)
    patients = [x for x in patients if x!= '.DS_Store']
    for patient in patients:
        for zoom in ZOOMS:
          zoom_path = patient_path + "/" + patient + "/" + zoom
          images = os.listdir(zoom_path)
          images = [x for x in images if x!= '.DS_Store']
          for image in images:
            copyfile(zoom_path + "/" + image, 
                     DATA_ROOT + "/" + zoom + "/" + class_type + "/" + image)
          

In [0]:
TRAIN_P = 0.8
TEST_P = 1 - TRAIN_P

for zoom in ZOOMS:
  split_folders.ratio(DATA_ROOT + "/" + zoom, 
                      output="breakhis/" + zoom, 
                      seed=6482, 
                      ratio=(TRAIN_P, TEST_P)) # default values

## zip and store

In [0]:
!tar -czvf breakhis.tar.gz breakhis

In [0]:
!cp breakhis.tar.gz "/content/gdrive/My Drive/colab_data/"

### All Data

In [0]:
!tar -xzvf "/content/gdrive/My Drive/colab_data/breakhis.tar.gz"

In [0]:
!mkdir -p breakhis_all/benign

In [0]:
!cp -r breakhis/*/*/benign breakhis_all/

In [0]:
!ls -l breakhis_all/benign/ | wc -l  

In [0]:
!cp -r breakhis/*/*/malignant breakhis_all/

In [0]:
!ls -l breakhis_all/malignant/ | wc -l  

In [0]:
!tar -czvf breakhis_all.tar.gz breakhis_all

In [0]:
!cp breakhis_all.tar.gz "/content/gdrive/My Drive/colab_data/"

In [0]:
TRAIN_P = 0.8
TEST_P = 1 - TRAIN_P

for zoom in ZOOMS:
  split_folders.ratio(DATA_ROOT + "/" + zoom, 
                      output="breakhis/" + zoom, 
                      seed=6482, 
                      ratio=(TRAIN_P, TEST_P)) # default values